<a href="https://colab.research.google.com/github/alexander-toschev/ml-cs-intro/blob/main/home-work/keras_homework_en_autograded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework: Introduction to Keras (with simple autograder)

In this homework you will work with `tf.keras` on the MNIST dataset.

Goals:
- recall the basic training pipeline for a neural network;
- build a simple Keras model (MLP and CNN);
- practice using callbacks and regularization;
- compare the performance of different architectures.

Each task has an assigned number of points.

**Maximum score: 100 points**  
Task 1 — 10 pts  
Task 2 — 20 pts  
Task 3 — 20 pts  
Task 4 — 20 pts  
Task 5 — 30 pts

Below is a **very simple autograder**:
- After each task there is a test cell.
- If tests pass, points for that task are added to the total.
- If tests fail or raise an error, the task gives 0 points.


In [ ]:
# @title 1) Student Info & Config
# All code comments are in English.


# === ОБЯЗАТЕЛЬНО ЗАПОЛНИТЬ ===
full_name = "Doe John"     # например: "Тощев Александр"
student_group = "11-111"      # например: "208"
assignment_id = "EM02_Keras"
assert full_name != "Фамилия Имя", "Заполните full_name"
assert student_group != "Группа", "Заполните student_group"
print("✔ Student Info OK")

# Typical human accuracy (benchmark) for MNIST may be ~97-99%.

print("Student:", full_name)

✔ Student Info OK
Student: Doe John


In [ ]:
import numpy as np

def _close(a, b, tol=1e-8):
    return np.allclose(a, b, atol=tol)

def _arr_equal(a, b):
    a = np.array(a)
    b = np.array(b)
    return a.shape == b.shape and np.array_equal(a, b)

In [ ]:
from datetime import datetime, timezone, timedelta

# Установите окна приёма (пример):

start_at_iso = "2025-12-01T09:00-04:00"  #@param {type:"string"}
due_at_iso   = "2025-12-08T23:59-04:00"  #@param {type:"string"}
start_dt = datetime.fromisoformat(start_at_iso)
due_dt   = datetime.fromisoformat(due_at_iso)
# Для протокола: время сдачи берём текущее (можно заменить на mtime файла)
import os
from datetime import datetime, timezone

# 📅 Add submission date based on file modification time
try:
    nb_path = __file__ if "__file__" in globals() else "EM01_Numpy.ipynb"
    mtime = os.path.getmtime(nb_path)
    submission_dt = datetime.fromtimestamp(mtime, tz=timezone.utc)
except Exception:
    submission_dt = datetime.utcnow().replace(tzinfo=timezone.utc)

def penalty_fraction(start_dt, due_dt, submission_dt):
    """Возвращает долю штрафа [0..1].
    0 — без штрафа (<= due_dt). Линейно растёт от due_dt к due_dt + (due_dt - start_dt).
    Не выходит за 1.0.
    """
    if submission_dt <= due_dt:
        return 0.0
    total = (due_dt - start_dt).total_seconds()
    late  = (submission_dt - due_dt).total_seconds()
    if total <= 0:
        return 1.0 if late > 0 else 0.0
    return min(1.0, max(0.0, late / total))

print(f"Окно приёма: {start_dt.isoformat()} — {due_dt.isoformat()} (UTC)")
print(f"Время сдачи: {submission_dt.isoformat()} (UTC)")

Окно приёма: 2025-11-24T09:00:00-04:00 — 2025-12-01T23:59:00-04:00 (UTC)
Время сдачи: 2025-12-01T14:07:03.072575+00:00 (UTC)


/tmp/ipython-input-632059888.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  submission_dt = datetime.utcnow().replace(tzinfo=timezone.utc)


In [ ]:
# === Grading setup ===

max_points = {
    "task1": 10,
    "task2": 20,
    "task3": 20,
    "task4": 20,
    "task5": 30,
}

total_points = 0

def add_points(task_name, ok):
    """Add points for a task if ok == True, print running total."""
    global total_points
    max_total = sum(max_points.values())
    if ok:
        pts = max_points[task_name]
        total_points += pts
        print(f"{task_name}: +{pts} points ✅ (total = {total_points}/{max_total})")
    else:
        print(f"{task_name}: 0 points ❌ (total = {total_points}/{max_total})")


In [ ]:
# If needed, uncomment the line below to install/upgrade TensorFlow
# !pip install -q "tensorflow>=2.15"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.19.0


## Task 1 (10 points)
### Loading and preprocessing data

Load the MNIST dataset using `keras.datasets.mnist.load_data()` and prepare the data:
- cast to `float32`;
- normalize pixel values to `[0, 1]` (divide by 255);
- print shapes of train and test arrays.

**Required:** implement `load_mnist()` that returns a tuple `(x_train, y_train, x_test, y_test)` with **normalized** data.


In [ ]:
def load_mnist():
    """Load and normalize MNIST.
    Returns: x_train, y_train, x_test, y_test.
    """
    # TODO: your code here
    # 1) load data
    # 2) cast to float32
    # 3) divide by 255
    raise NotImplementedError


In [ ]:
# === Autograder: Task 1 ===
try:
    x_train, y_train, x_test, y_test = load_mnist()
    cond_dtype = (x_train.dtype == np.float32 and x_test.dtype == np.float32)
    cond_range = (
        x_train.min() >= -1e-6 and x_train.max() <= 1.0 + 1e-6 and
        x_test.min() >= -1e-6 and x_test.max() <= 1.0 + 1e-6
    )
    cond_shape = (
        x_train.shape[0] == y_train.shape[0] and
        x_test.shape[0] == y_test.shape[0]
    )
    ok = bool(cond_dtype and cond_range and cond_shape)
    add_points("task1", ok)
except Exception as e:
    print("Task 1 error:", e)
    add_points("task1", False)


Task 1 error: 
task1: 0 points ❌ (total = 0/100)


## Task 2 (20 points)
### Basic MLP model in Keras

Implement `build_mlp(input_shape, num_classes)` that creates and returns a **compiled** Keras model.

Architecture requirements (you may add layers, but do not simplify):
- input: `Input(shape=input_shape)`;
- `Flatten()` layer;
- dense layer with **at least 64 units** and `relu` activation;
- `Dropout` layer with rate at least 0.2;
- output: Dense with `num_classes` units and `softmax` activation.

Compilation requirements:
- optimizer: Adam;
- loss: `sparse_categorical_crossentropy`;
- metric: accuracy.


In [ ]:
def build_mlp(input_shape, num_classes):
    """Create and compile a simple MLP model in Keras.

    input_shape: tuple, e.g. (28, 28)
    num_classes: number of classes (for MNIST — 10)
    """
    # TODO: your code here
    # 1) build a Sequential model
    # 2) add Flatten, Dense, Dropout, Dense
    # 3) call compile(...)
    raise NotImplementedError


In [ ]:
# === Autograder: Task 2 ===
try:
    model_mlp = build_mlp((28, 28), 10)
    compiled = hasattr(model_mlp, "loss") and model_mlp.loss is not None
    layer_types = [type(l).__name__ for l in model_mlp.layers]
    has_flatten = "Flatten" in layer_types
    has_dropout = "Dropout" in layer_types
    has_dense = layer_types.count("Dense") >= 2
    ok = bool(compiled and has_flatten and has_dropout and has_dense)
    add_points("task2", ok)
except Exception as e:
    print("Task 2 error:", e)
    add_points("task2", False)


Task 2 error: 
task2: 0 points ❌ (total = 0/100)


## Task 3 (20 points)
### Training with validation and callbacks

Implement `train_model(model, x_train, y_train, x_val, y_val)` that:
- takes a **compiled** model;
- trains it on `x_train, y_train`;
- uses validation data `x_val, y_val`;
- returns the `history` object.

Requirements:
- `batch_size` not larger than 256;
- at least 5 epochs of training;
- use `EarlyStopping` callback on validation loss with `patience` between 2 and 4 and `restore_best_weights=True`.


In [ ]:
def train_model(model, x_train, y_train, x_val, y_val):
    """Train the model and return history.

    Configure EarlyStopping here.
    """
    # TODO: your code here
    # 1) create EarlyStopping callback
    # 2) call model.fit(...)
    raise NotImplementedError


In [ ]:
# === Autograder: Task 3 ===
try:
    x_train, y_train, x_test, y_test = load_mnist()
    x_tr, x_val = x_train[:5000], x_train[5000:6000]
    y_tr, y_val = y_train[:5000], y_train[5000:6000]

    model_mlp = build_mlp((28, 28), 10)
    history_mlp = train_model(model_mlp, x_tr, y_tr, x_val, y_val)

    h = getattr(history_mlp, "history", {})
    ok = all(k in h for k in ["loss", "val_loss", "accuracy", "val_accuracy"])
    add_points("task3", bool(ok))
except Exception as e:
    print("Task 3 error:", e)
    add_points("task3", False)


Task 3 error: 
task3: 0 points ❌ (total = 0/100)


## Task 4 (20 points)
### Plotting training curves

Implement `plot_history(history)` that:
- plots loss (train/val) vs epoch;
- plots accuracy (train/val) vs epoch.

Requirements:
- use `matplotlib`;
- add legend and axis labels.


In [ ]:
def plot_history(history):
    """Plot training curves from a Keras history object."""
    # TODO: your code here
    # Hint: history.history is a dict with keys 'loss', 'val_loss', 'accuracy', 'val_accuracy'
    raise NotImplementedError


In [ ]:
# === Autograder: Task 4 ===
try:
    plot_history(history_mlp)
    add_points("task4", True)
except Exception as e:
    print("Task 4 error:", e)
    add_points("task4", False)


Task 4 error: name 'history_mlp' is not defined
task4: 0 points ❌ (total = 0/100)


## Task 5 (30 points)
### CNN for MNIST and comparison with MLP

Now you need to:
1. Implement `build_cnn(input_shape, num_classes)` that builds a simple **convolutional** model:
   - at least two convolutional layers (`Conv2D`) with `relu` activation;
   - pooling (e.g. `MaxPooling2D`);
   - `Flatten` layer;
   - one or two dense layers and final `Dense(num_classes, softmax)`.
2. Remember that for a CNN the input shape should be `(H, W, 1)` — you need to add a channel dimension.
3. Train the CNN on the same data as the MLP (you can reuse `train_model`).
4. Compare test accuracy for both models (print both values).

**Required:**
- implement `build_cnn`;
- properly reshape the data;
- train the CNN and print final test accuracy.


In [ ]:
def build_cnn(input_shape, num_classes):
    """Create and compile a simple CNN model for MNIST."""
    # TODO: your code here
    # 1) use Conv2D, MaxPooling2D, Flatten, Dense
    # 2) don't forget compile(...)
    raise NotImplementedError


In [ ]:
# === Autograder: Task 5 ===
try:
    x_train, y_train, x_test, y_test = load_mnist()
    x_train_cnn = np.expand_dims(x_train, axis=-1)
    x_test_cnn = np.expand_dims(x_test, axis=-1)

    x_tr_cnn, x_val_cnn = x_train_cnn[:5000], x_train_cnn[5000:6000]
    y_tr_cnn, y_val_cnn = y_train[:5000], y_train[5000:6000]

    model_cnn = build_cnn((28, 28, 1), 10)
    history_cnn = train_model(model_cnn, x_tr_cnn, y_tr_cnn, x_val_cnn, y_val_cnn)

    layer_types = [type(l).__name__ for l in model_cnn.layers]
    has_conv = layer_types.count("Conv2D") >= 2
    has_pool = "MaxPooling2D" in layer_types
    ok = bool(has_conv and has_pool)
    add_points("task5", ok)
except Exception as e:
    print("Task 5 error:", e)
    add_points("task5", False)


Task 5 error: 
task5: 0 points ❌ (total = 0/100)


## Final total

Run the cell below at the end just to re-print the final score stored in `total_points`.


In [ ]:
max_total = sum(max_points.values())
print(f"Final score: {total_points}/{max_total}")


Final score: 0/100


In [ ]:
import json
import numpy as np
from datetime import datetime, timezone

# ---------- тестовые помощники ----------
def add_points(ok, pts, msg_ok, msg_fail):
    global total
    if ok:
        total += pts
        print(f"[+{pts:>2}] {msg_ok}")
    else:
        print(f"[ 0] {msg_fail}")

def _sec(td):
    return td.total_seconds()

def penalty_fraction(start_dt, due_dt, now_dt) -> float:
    if not (start_dt and due_dt and now_dt):
        return 0.0
    window = _sec(due_dt - start_dt)
    if window <= 0:
        return 1.0 if now_dt > due_dt else 0.0
    late = max(0.0, _sec(now_dt - due_dt))
    return min(1.0, late / window)

# ---------- автогрейдинг ----------
total = total_points
max_points = max_total
raw_score = total_points

import json

# применяем штраф
try:
    pf = penalty_fraction(start_dt, due_dt, submission_dt)
except NameError:
    from datetime import timezone
    pf = 0.0
# ✅ Итоговый результат

max_points = 100

raw_score = round(100.0 * total / max_points)
final_score = max(0.0, raw_score * (1.0 - min(1.0, pf)))

print(f"Сырой балл: {raw_score}/{max_points}")
print(f"Штраф (доля): {pf:.4f}")
print(f"Итоговый балл после штрафа: {final_score:.2f}/{max_points}")

# Последняя строка — JSON, который читает harness
final = {
    "name": full_name,
    "group": student_group,
    "assignment": assignment_id,
    "score": float(final_score)
}

Сырой балл: 0/100
Штраф (доля): 0.0000
Итоговый балл после штрафа: 0.00/100


In [ ]:
print(json.dumps(final, ensure_ascii=False))

{"name": "Doe John", "group": "11-111", "assignment": "EM01_Numpy", "score": 0.0}
